In [6]:
import pandas as pd
import numpy as np
from numpy.linalg import svd


df = pd.read_csv('ratings_small.csv')
df.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


Precisamos passar todos os dados para uma matriz

Exemplo 3 usuarios e 4 filmes

usuario1 = [nota_1_1, nota_1_2, nota_1_3, nota_1_4]

usuario2 = [nota_2_1, nota_2_2, nota_2_3, nota_2_4]

usuario3 = [nota_3_1, nota_3_2, nota_3_3, nota_3_4]


portanto nossa matiz A ficaria assim:

$$
A = \begin{bmatrix}
    nota_{1,1} & nota_{1,2} & nota_{1,3} & nota_{1,4} \\
    nota_{2,1} & nota_{2,2} & nota_{2,3} & nota_{2,4} \\
    nota_{3,1} & nota_{3,2} & nota_{3,3} & nota_{3,4} \\
\end{bmatrix}



In [25]:
'''
Exemplo 3 usuarios e 4 filmes

usuario1 = [nota_1_1, nota_1_2, nota_1_3, nota_1_4]
usuario2 = [nota_2_1, nota_2_2, nota_2_3, nota_2_4]
usuario3 = [nota_3_1, nota_3_2, nota_3_3, nota_3_4]

matriz A = [ [nota_1_1, nota_1_2, nota_1_3, nota_1_4],
                         [nota_2_1, nota_2_2, nota_2_3, nota_2_4],
                         [nota_3_1, nota_3_2, nota_3_3, nota_3_4]]
'''

# Criando a matriz de notas
matriz_notas = df.pivot_table(index='userId', columns='movieId', values='rating')
matriz_notas = matriz_notas.fillna(0)
matriz_notas.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.decomposition import TruncatedSVD

def svd_input(A, n_components):
    # Calculando a matriz de notas médias
    notas_medias = np.nanmean(A, axis=1)
    A_demeaned = A - notas_medias.reshape(-1, 1)

    # Calculando a SVD truncada
    svd = TruncatedSVD(n_components=n_components)
    svd.fit(A_demeaned)
    U = svd.transform(A_demeaned)
    sigma = svd.singular_values_
    Vt = svd.components_

    return U, sigma, Vt, notas_medias


In [39]:
U, sigma, Vt, notas_medias = svd_input(matriz_notas.values, 500) # quanto maior o n_components, mais precisa a recomendação

A_reconstruida = U @ np.diag(sigma) @ Vt + notas_medias.reshape(-1, 1)

# Calculando o erro
erro = np.sum((matriz_notas.values - A_reconstruida)**2)
print(f'Erro: {erro}')

# Primeiro componente principal (vetor coluna de U)
cp1 = U[:, 0]

# Segundo componente principal (vetor coluna de Vt)
cp2 = Vt.T[:, 1]

# Variância explicada pelos dois primeiros componentes principais
var_exp = (sigma[0]**2 + sigma[1]**2) / np.sum(sigma**2)
print(f'Variância explicada pelos dois primeiros CPs: {var_exp:.2%}')

Erro: 55857678379.88517
Variância explicada pelos dois primeiros CPs: 21.23%


In [42]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculando a similaridade entre os usuários
similaridade_U = cosine_similarity(U)
similaridade_U

# Calculando a similaridade entre os filmes
similaridade_F = cosine_similarity(Vt.T)
similaridade_F


array([[ 1.00000000e+00,  3.04020647e-02,  7.11137340e-02, ...,
        -7.37588057e-03, -7.41523432e-03,  8.58075711e-04],
       [ 3.04020647e-02,  1.00000000e+00,  3.89037519e-03, ...,
        -8.35771728e-04, -1.99060695e-03,  9.89788781e-03],
       [ 7.11137340e-02,  3.89037519e-03,  1.00000000e+00, ...,
         2.78050126e-02,  2.74809186e-02, -6.44775164e-03],
       ...,
       [-7.37588057e-03, -8.35771728e-04,  2.78050126e-02, ...,
         1.00000000e+00,  9.99432237e-01, -2.69463731e-02],
       [-7.41523432e-03, -1.99060695e-03,  2.74809186e-02, ...,
         9.99432237e-01,  1.00000000e+00, -2.97797861e-02],
       [ 8.58075711e-04,  9.89788781e-03, -6.44775164e-03, ...,
        -2.69463731e-02, -2.97797861e-02,  1.00000000e+00]])

In [44]:
# Índices dos filmes já assistidos pelo usuário 1
indices_assistidos = matriz_notas.loc[1][matriz_notas.loc[1] > 0].index

# Similaridade entre o usuário 1 e todos os outros usuários
similaridade_usuarios = cosine_similarity(U[1].reshape(1, -1), U)[0]

# Notas médias dadas por cada usuário aos filmes assistidos pelo usuário 1
notas_medias_assistidos = matriz_notas.loc[:, indices_assistidos].mean()

# Cálculo das notas dos filmes ainda não assistidos pelo usuário 1
notas_filmes_nao_assistidos = np.zeros(matriz_notas.shape[1])

for i in range(matriz_notas.shape[1]):
    if i not in indices_assistidos:
        # Notas médias dadas por cada usuário aos filmes ainda não assistidos pelo usuário 1
        notas_medias_nao_assistidos = matriz_notas.iloc[:, i].mean()
        
        # Similaridade entre o usuário 1 e todos os usuários que assistiram ao filme i
        similaridade_filme = cosine_similarity(Vt.T[i].reshape(1, -1), U)[0]
        
        # Notas dadas pelos usuários ao filme i, ponderadas pela similaridade com o usuário 1
        notas_ponderadas = (matriz_notas.iloc[:, i] - notas_medias_nao_assistidos) * similaridade_filme * similaridade_usuarios
        notas_filmes_nao_assistidos[i] = notas_medias_nao_assistidos + notas_ponderadas.sum() / (similaridade_filme * similaridade_usuarios).sum()

# Índices dos 10 filmes com maior nota
indices_recomendados = np.argsort(notas_filmes_nao_assistidos)[-10:][::-1]

# Notas dos 10 filmes com maior nota
notas_recomendadas = notas_filmes_nao_assistidos[indices_recomendados]


In [46]:
# Obter índices dos filmes ainda não assistidos pelo usuário 1
indices_nao_assistidos = [i for i in range(matriz_notas.shape[1]) if i not in indices_assistidos]

# Calcular notas médias dos filmes ainda não assistidos pelo usuário 1
notas_medias_nao_assistidos = matriz_notas.iloc[:, indices_nao_assistidos].mean()

# Obter as similaridades entre o filme 1 e os filmes ainda não assistidos
similaridades_filme1 = similaridade_F[0, indices_nao_assistidos]

# Calcular as notas ponderadas dos filmes ainda não assistidos
notas_ponderadas = np.dot(similaridades_filme1, matriz_notas.iloc[0, indices_nao_assistidos] - notas_medias_nao_assistidos) / np.sum(np.abs(similaridades_filme1))

# Obter as notas previstas adicionando a nota média dos filmes ainda não assistidos
notas_previstas = notas_ponderadas + notas_medias_nao_assistidos

# Criar um DataFrame com os resultados
resultados = pd.DataFrame({'filme': matriz_notas.columns[indices_nao_assistidos], 'nota': notas_previstas})

# Ordenar os resultados em ordem decrescente de nota prevista
resultados = resultados.sort_values('nota', ascending=False)

# Obter os 10 filmes com maior nota prevista
resultados.head(10)


,filme,nota
movieId,,
318,318,2.065881
356,356,2.046507
296,296,2.041291
593,593,1.860963
260,260,1.816999
2571,2571,1.600904
527,527,1.550978
480,480,1.499562
1196,1196,1.462304
